# Example usage for connection to development Docker-based cluster

This will allow you to connect from your dev machine to the cluster with Spark Connect.

The `data` directory is mapped to `/opt/spark-data` on the Spark machines.  When you create a table it will be stored in `data/spark-warehouse` which is common to your machine.

## Installation

You need to make sure you virtual Python environment has the dependencies listed in `automation/spark-connect-requirements.txt`  (install with `python -m pip install -r spark-connect-requirements.txt`).

NB:  Make sure the version of pyspark in your local environment is same as the `build/spark.Dockerfile` version.  Also Spark 3.5.x had multiple problems with the worker nodes connecting so downgraded to 3.4.2 at the moment.

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("TestA") \
    .remote("sc://localhost:15002") \
    .getOrCreate()

In [9]:
df = spark.read.format("csv").option("header",True).load("/opt/spark-data/insurance_claims_report.csv")

In [10]:
df.count()

1000

In [11]:
df.toPandas()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
0,328,48,521585,17/10/14,OH,250/500,1000,1406.91,0,466132,...,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y
1,228,42,342868,27/06/06,IN,250/500,2000,1197.22,5000000,468176,...,0,?,5070,780,780,3510,Mercedes,E400,2007,Y
2,134,18,687698,06/09/00,OH,100/300,2000,1413.14,5000000,430632,...,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N
3,256,41,227811,25/05/90,IL,250/500,2000,1415.74,6000000,608117,...,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y
4,228,44,367455,06/06/14,IL,500/1000,1000,1583.91,6000000,610706,...,1,NO,6500,1300,650,4550,Accura,RSX,2009,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,38,941851,16/07/91,OH,500/1000,1000,1310.8,0,431289,...,1,?,87200,17440,8720,61040,Honda,Accord,2006,N
996,285,41,186934,05/01/14,IL,100/300,1000,1436.79,0,608177,...,3,?,108480,18080,18080,72320,Volkswagen,Passat,2015,N
997,130,34,918516,17/02/03,OH,250/500,500,1383.49,3000000,442797,...,3,YES,67500,7500,7500,52500,Suburu,Impreza,1996,N
998,458,62,533940,18/11/11,IL,500/1000,2000,1356.92,5000000,441714,...,1,YES,46980,5220,5220,36540,Audi,A5,1998,N


In [21]:
df.head(1)

[Row(months_as_customer='328', age='48', policy_number='521585', policy_bind_date='17/10/14', policy_state='OH', policy_csl='250/500', policy_deductable='1000', policy_annual_premium='1406.91', umbrella_limit='0', insured_zip='466132', insured_sex='MALE', insured_education_level='MD', insured_occupation='craft-repair', insured_hobbies='sleeping', insured_relationship='husband', capital-gains='53300', capital-loss='0', incident_date='25/01/15', incident_type='Single Vehicle Collision', collision_type='Side Collision', incident_severity='Major Damage', authorities_contacted='Police', incident_state='SC', incident_city='Columbus', incident_location='9935 4th Drive', incident_hour_of_the_day='5', number_of_vehicles_involved='1', property_damage='YES', bodily_injuries='1', witnesses='2', police_report_available='YES', total_claim_amount='71610', injury_claim='6510', property_claim='13020', vehicle_claim='52080', auto_make='Saab', auto_model='92x', auto_year='2004', fraud_reported='Y')]

In [19]:
df.write.saveAsTable("default.insurance_claims")

In [20]:
spark.catalog.listTables("default")

[Table(name='insurance_claims', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='testa', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False)]